Colab에서 R 사용 가능

[링크 텍스트](https://www.youtube.com/watch?v=ugNlbyOlWlQ) #이 분 없었으면 평생 헤맸을듯

In [1]:
#엑셀 불러올 준비
system("gdown --id 1q0JuMeYNscfq2BrGuEfk-_fDZTj2y5k-")
library(readxl)

In [28]:
df_master = read_excel("df_master.xlsx") #구글 드라이브에서 주소 가져올 때 링크가 있는 모든 사용자에게 공개 꼭! 해줘야함 

In [3]:
unique(df_master$CLAC1_NM)

[1] "건강식품" "냉동식품" "냉장식품" "과일"     "축산물"

In [4]:
length(unique(df_master$CLNT_ID)) # 고유 ID는 9만 33397명

[1] 93397

In [38]:
library(haven)
df_master$MONTH <- format(as.Date(df_master$SESS_DT), "%m")
df_master$MONTH <- as.integer(df_master$MONTH)

In [44]:
pdf_master <- pdata.frame(df_master, index=c("CLNT_ID","MONTH"))

#CLNT_ID와 함께 시간번수를 index로 넣으려고 했으나 오류 발생 
#아마도 SESS_DT가 POSIX-format(날짜형식)이라서 그런 것 같음. 

Warning message in pdata.frame(df_master, index = c("CLNT_ID", "MONTH")):
“duplicate couples (id-time) in resulting pdata.frame
 to find out which, use, e.g., table(index(your_pdataframe), useNA = "ifany")”


In [9]:
#SESS_DT 형식으로 인해 index에 추가하지 못하는 오류 
# 참고: https://stackoverflow.com/questions/31773723/r-plm-regression-with-time-in-posix-format
#install.packages('plm')
library(dplyr)
library(plm)
df_master <- df_master %>%
    group_by(CLNT_ID) %>%
    mutate(clock = seq_along(CLNT_ID))
panel <- pdata.frame(df_master, c("CLNT_ID", "SESS_DT"))


Attaching package: ‘plm’


The following objects are masked from ‘package:dplyr’:

    between, lag, lead


Warning message in pdata.frame(df_master, c("CLNT_ID", "SESS_DT")):
“duplicate couples (id-time) in resulting pdata.frame
 to find out which, use, e.g., table(index(your_pdataframe), useNA = "ifany")”


In [29]:
df_master_health <- subset(df_master, CLAC1_NM=="건강식품") 
df_master_ice <- subset(df_master, CLAC1_NM=="냉동식품") 
df_master_ref <- subset(df_master, CLAC1_NM=="냉장식품") 
df_master_fruit <- subset(df_master, CLAC1_NM=="과일")
df_master_meet  <- subset(df_master, CLAC1_NM=="축산물")
#따로 분석해볼까 싶어서 분리

In [30]:
df_master$DT_INT <- as.integer(format(df_master$SESS_DT, "%Y%m%d"))

In [33]:
panel <- pdata.frame(df_master, c("CLNT_ID", "SESS_DT"))

Warning message in pdata.frame(df_master, c("CLNT_ID", "SESS_DT")):
“duplicate couples (id-time) in resulting pdata.frame
 to find out which, use, e.g., table(index(your_pdataframe), useNA = "ifany")”


In [45]:
a <- plm(log(PD_AMOUNT) ~ factor(SEASON)+TEMP_AVG+RAINFALL_DAY+HUMID_AVG+DUST, model="within", data=pdf_master)

Warning message:
“non-unique values when setting 'row.names': ‘100004-5’, ‘1000190-5’, ‘1000453-9’, ‘1000657-5’, ‘1000699-4’, ‘1000758-9’, ‘1000974-8’, ‘1001157-7’, ‘1001544-4’, ‘1001840-5’, ‘1002049-5’, ‘1002452-9’, ‘1002487-6’, ‘1002618-8’, ‘1002713-6’, ‘1002833-7’, ‘1002833-8’, ‘1003050-9’, ‘1003684-8’, ‘100392-7’, ‘100400-5’, ‘100419-9’, ‘1004249-8’, ‘10043-9’, ‘1004780-7’, ‘1004909-4’, ‘1004992-9’, ‘1005008-4’, ‘1005070-9’, ‘1005110-9’, ‘1005236-8’, ‘1005315-5’, ‘1005549-4’, ‘1005606-9’, ‘1005666-5’, ‘1005925-8’, ‘1006217-9’, ‘1006502-6’, ‘1006663-9’, ‘1007384-7’, ‘1007525-7’, ‘1007880-5’, ‘1007910-8’, ‘1008017-9’, ‘1008164-9’, ‘10082-8’, ‘100826-4’, ‘1008306-7’, ‘100842-8’, ‘1008786-4’, ‘1008824-7’, ‘1008991-9’, ‘1009130-4’, ‘1009160-8’, ‘1009261-9’, ‘1009293-7’, ‘100952-5’, ‘10097-9’, ‘1009717-7’, ‘1009735-8’, ‘1009793-4’, ‘1009793-5’, ‘1009793-6’, ‘1009866-5’, ‘1009866-7’, ‘1009934-4’, ‘1009957-4’, ‘10101-5’, ‘101019-4’, ‘1010286-4’, ‘1010286-5’, ‘1010311-9’, ‘1010738-9’, ‘1011

ERROR: ignored

In [46]:
b <- plm(log(PD_AMOUNT) ~ factor(SEASON)+TEMP_AVG+RAINFALL_DAY+HUMID_AVG+DUST, index=c("CLNT_ID"), data=df_master, model="random")
summary(b)
#와씨 r제곱 0.86

Oneway (individual) effect Within Model

Call:
plm(formula = log(PD_AMOUNT) ~ factor(SEASON) + TEMP_AVG + RAINFALL_DAY + 
    HUMID_AVG + DUST, data = df_master, model = "within", index = c("CLNT_ID"))

Unbalanced Panel: n = 93397, T = 1-576, N = 188189

Residuals:
     Min.   1st Qu.    Median   3rd Qu.      Max. 
-5.671406 -0.067130  0.000000  0.055415  3.871861 

Coefficients:
                        Estimate  Std. Error  t-value  Pr(>|t|)    
factor(SEASON)spring -0.19010405  0.01413507 -13.4491 < 2.2e-16 ***
factor(SEASON)summer -0.17314923  0.01283875 -13.4865 < 2.2e-16 ***
TEMP_AVG             -0.00363611  0.00074383  -4.8884 1.018e-06 ***
RAINFALL_DAY          0.00046972  0.00017817   2.6364  0.008381 ** 
HUMID_AVG             0.00020616  0.00021658   0.9519  0.341156    
DUST                  0.00051156  0.00019180   2.6671  0.007653 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    30702
Residual Sum of Squares: 30616
R-Squared: 

# 패널 회귀분석 공부좀 해야됨 
## 고정효과, 임의효과 개념 아직도 제대로 모름ㅠ 

